<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [34]:
import tensorflow as tf
tf.variable_scope = tf.compat.v1.variable_scope
import numpy as np
from cgraphviz import *  
from tfhelper import *
from ckpt_hookers import *
from dataloader import *
import argparse
import importlib

In [35]:
dataload = Sass('/home/penalvad/stattus4/benchfull/', num_samples=10, number_of_batches=10,split_tax=0.8, freq_size=400, time_size=35, same_data_validation=True)

In [36]:
dataload.sampled_batches

[0, 0]

In [ ]:
# enable tf logging, show DEBUG output

tf.logging.set_verbosity(tf.logging.DEBUG)
'''
parser = argparse.ArgumentParser()

parser.add_argument(
    "--datadir", type=str, default="/home/penalvad/stattus4/benchfull",
    help="data directory"
    )

# model
parser.add_argument(
    "--model", type=str, default="model.gcnn",
    help="model used"
    )

# training
parser.add_argument(
    "--dataset", type=str, default="data.benchfull",
    help="dataset used"
    )


parser.add_argument(
    "--epoch", type=int, default=300,
    help="training epochs"
    )

parser.add_argument(
    "--batchsize", type=int, default=100,
    help="batch size"
)

# logging / saving
parser.add_argument(
    "--logdir", type=str, default="./",
    help="log folder"
    )

parser.add_argument(
    "--save_mins", dest="save_mins", default=5, type=int,
    help="""Save the graph and summaries of once every N steps."""
)

parser.add_argument(
    "--log_steps", dest="log_steps", default=10, type=int,
    help="""Log the values of once every N steps."""
)

args = parser.parse_args()
tf.summary.text("settings", tf.constant(" ".join(["| **" + i + "** | `" + str(k) + "` |" for i, k in vars(args).items()
'''

        # add description text to your log
tf.add_to_collection("SUMMARIES_ONCE", tf.summary.text("settings", tf.constant('settings foo'), collections="SUMMARIES_ONCE"))

In [ ]:
graph = tf.Graph()
with graph.as_default():
  with tf.variable_scope('signal_in'):
    signal_in = tf.placeholder(tf.float32, shape=(10,40,2,1))

  with tf.variable_scope('dascope1'):
    conv_linear = tf.keras.layers.Conv2D( 8, (8,2), padding='valid', name='conv_linear', use_bias=True, kernel_initializer=tf.initializers.lecun_normal(seed=137), bias_initializer=tf.initializers.lecun_normal(seed=137) )(signal_in)
  
  
  with tf.variable_scope('softmax'):
    logits = tf.contrib.layers.fully_connected(conv_linear, 2, activation_fn=None, normalizer_fn=None, normalizer_params=None, weights_initializer=tf.initializers.lecun_normal(seed=731), weights_regularizer=None, biases_initializer=tf.initializers.lecun_normal(seed=777), biases_regularizer=None, reuse=None, variables_collections=None, outputs_collections=None, trainable=True, scope='logit')
    softmax = tf.nn.softmax(logits,axis=0)            
    
  with tf.variable_scope('loss'):
    l_vec = tf.placeholder(tf.float32, shape=(10,2))
    globalstep = tf.compat.v1.train.get_or_create_global_step()        
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0)(l_vec, softmax)         
    minimize_op = tf.train.AdamOptimizer(learning_rate=0.05).minimize(loss, global_step = globalstep)    
    initop = tf.global_variables_initializer()
    graph.add_to_collections(tf.GraphKeys.INIT_OP, initop)
    
print_display_cgraph(graph)    
sess = tf.Session(graph=graph)
lvec = [[1,0],[0,1],[1,0],[0,1],[1,0],[0,1],[1,0],[0,1],[1,0],[0,1]]
'''
with sess.as_default():
    
  feed_dict = {
      signal_in:np.random.rand(10,40,2,1),
      l_vec: lvec
  }
    
  sess.run(graph.get_collection_ref(tf.GraphKeys.INIT_OP)[0])
  for i in range(2):
    lossvalue, _ = sess.run([loss, minimize_op], feed_dict=feed_dict)
    print(lossvalue)  
'''

saver_hook = tf.estimator.CheckpointSaverHook('./ckpt', save_steps=2, checkpoint_basename='experiment.ckpt')
with tf.train.MonitoredTrainingSession(hooks=[saver_hook], checkpoint_dir='checkpoint') as sess:

  feed_dict = {
    signal_in:np.random.rand(10,40,2,1),
    l_vec: lvec
  }  

  while not sess.should_stop():  
        
    step = sess.run(globalstep)
    if (step % 10 == 0):
      print(step)
      lossvalue, _ = sess.run([loss, minimize_op], feed_dict=feed_dict)
      print(lossvalue)  

In [ ]:
#desambiguatename(opnamescope, scope)
#get_listeners(graph, scope='loss/init')
op = get_op_fromscope(graph, scope='dascope1/', opname='Conv2D')
op = op[0]
graphdef = tf.tools.graph_transforms.TransformGraph( graph.as_graph_def(), [], [], ['remove_nodes(op=init)'])

with tf.Graph().as_default() as g:  
  tf.import_graph_def(graphdef,name = '')
  for op in g.get_operations():
    if op.name.split('/')[-1] == 'init':
      print('True')

In [ ]:
def copy_all(graph, copy_graph):
  for graph_key in graph.collections:
    for val in graph.get_collection_ref(graph_key): 
      copy_graph.add_to_collections(str(graph_key),val)

def copy_vars(graph, copy_graph, collection = 'trainable_variables'):
    
  for tensor in graph.get_collection_ref(collection):
    
    l = tensor.name.split('/')[:-1]
    name = ''
    for i in range(len(l)):
      if i == 0:
        name = name+l[i]
      else:
        name = name+'/'+l[i]
        
        
    if collection == tf.GraphKeys.TRAIN_OP:    
      copy_graph.add_to_collections(tf.GraphKeys.TRAIN_OP, tensor)    
    else:
      with tf.variable_scope(name, auxiliary_name_scope=False):
        tf.get_variable(tensor.name.split(':')[0].split('/')[-1], initializer=copy_graph.get_tensor_by_name(tensor.name), trainable=True) 

def strip_op(graph, opname, **kwargs):
  '''
   Returns the same graph without given op
  '''
  try:
    inputops = kwargs['input_ops']
  except:
    inputops = False
    
  nodes_after_strip = []
  #print(graph.as_graph_def().node)
  gd = graph.as_graph_def()
  op = graph.get_operation_by_name(opname)
  outputs = list(op.outputs)
  for node in gd.node:

    if node.name.startswith(opname) == False:
      copy_node = node_def_pb2.NodeDef()
      copy_node.CopyFrom(node)
    
      # Is the copied node an Output node to the given op ?
      for tensorout in outputs:        
        outmark = tensorout.name.find(':')   
        if copy_node.name == tensorout.name[:outmark]:
          new_input = []
        
          #Setup for the inputs of the copied nodes
          for node_name in copy_node.input:
      
            if node_name.startswith(op.name):
              if inputops != False:
                new_input.append(inputops)
              else:
                new_input.append(node_name)
        
        del copy_node.input[:]
        copy_node.input.extend(new_input)        
      nodes_after_strip.append(copy_node)   
    
  output_graphdef = graph_pb2.GraphDef()
  output_graphdef.node.extend(nodes_after_strip)
  return output_graphdef

## FUTURE: strip scope from graph
def strip_scope(graph, drop_scope, inputscope = '', ):
  '''
    Trashy given scope from graph and return the new graph.
  '''

  #
  ##  Output of dropped scope setup
  #  Output of another namescope is, by standart transformed into placeholder
  #  It returns a dict issuing the new placeholder in a new namescope named after signal_in 
    
  ## Input of dropped scope setup
  # Output of the namescope that feeds the dropped scope turnout to be a Variable. 
   
  for node in graph.node:
    input_nodes = graph.node
    new_node = node_def_pb2.NodeDef()
    new_node.CopyFrom(node)
    nodes_after_strip = []  
    

In [ ]:
# bagulho experimental doido
with tf.Graph().as_default() as graph0:
  print_display_cgraph(graph)     
  print('Antes Graph')
  for i in range(len(graph.collections)):
    print(10*'*')
    print(graph.collections[i])  
    print(graph.get_collection_ref(graph.collections[i]))    
  tf.import_graph_def(strip_op(graph,'loss/init'),name='')
  
  print_display_cgraph(graph0)    
  print('Antes Graph0')
  for i in range(len(graph0.collections)):
    print(10*'*')
    print(graph0.collections[i])  
    print(graph0.get_collection_ref(graph0.collections[i]))      
    
  copy_all(graph, graph0)  
  graph0.building_function
  #copy_vars(graph, graph0, collection=tf.GraphKeys.TRAIN_OP)          
  print_display_cgraph(graph0)    
  print('Depois Graph0')
  for i in range(len(graph0.collections)):
    print(10*'*')
    print(graph0.collections[i])  
    print(graph0.get_collection_ref(graph0.collections[i]))    
    
  #with tf.variable_scope('loss', auxiliary_name_scope=False):  
   # opini = tf.get_variable(graph0.get_collection_ref(tf.GraphKeys.INIT_OP)[0].name)
print('Depois Graph')
for i in range(len(graph.collections)):
  print(10*'*')
  print(graph.collections[i])  
  print(graph.get_collection_ref(graph.collections[i]))
      
sess = tf.Session(graph=graph0)

with sess.as_default():
    
  feed_dict = {
      graph0.get_tensor_by_name('signal_in/Placeholder:0'):np.random.rand(10,40,2,1),
      graph0.get_tensor_by_name('loss/Placeholder:0'): lvec
  }
  minimize_op = graph0.get_operation_by_name('loss/Adam')
  loss = graph0.get_tensor_by_name('loss/categorical_crossentropy/weighted_loss/value:0')  
  
  sess.run(graph0.get_collection_ref(tf.GraphKeys.INIT_OP))
  for i in range(2):
    print(i)
    lossvalue, _ = sess.run([loss, minimize_op], feed_dict=feed_dict)
    #print(lossvalue)   '''

In [ ]:
# oa
graph0 = tf.Graph()

with graph0.as_default():
  with tf.variable_scope('LosDiezMandamientos'):
    tf.get_variable('IO', dtype=tf.float32, shape=(1), initializer=None, regularizer=None, trainable=True)
    tf.global_variables_initializer()